In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk
from os import environ, getenv, path
from typing import TypedDict, Dict, Union, List
from simulator_funcs import *
import warnings

warnings.filterwarnings("ignore")

from odp.geospatial.odp_vessel_simulator.models.icct.database_functions.lookup_ship_data import fetch_ship_data
from odp.geospatial.odp_vessel_simulator.models.icct.database_functions.database import get_connection_pool, get_engine
import typing


## Color code explanation
<p style="color:#FE774C; font-size:12pt;"> Red means that values <i>have</i> to be inserted.</p> 
<p style="color:#9D59F4; font-size:12pt"> Purple means that values <i>can</i> be inserted if other that the default values are needed.</p> 


<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert all the mmsis for the vessels interest in here </p> 
<p> For example like the list under with two vessels (change the mmsis and add more of them if needed). </p> 

In [2]:
mmsis = [212552000, 319009700, 352297000,477630400]

<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert the names of the ports in the simulated paths </p> 
<p> For example like the list under where the path will go from Oslo to Kiel to Amsterdam. </p> 

In [3]:
ports = ['oslo','kiel','amsterdam']

In [4]:
df_ports, coordinates = get_lon_lat_ports(ports)

In [5]:
df_ports

,port_name,country,longitude,latitude
0,OSLO,NO,10.750000,59.916667
1,KIEL,DE,10.133333,54.316667
2,AMSTERDAM,NL,4.900000,52.366667


In [6]:
coordinates

[[10.75, 59.916667], [10.133333, 54.316667], [4.9, 52.366667]]

#### If the port does not exist, the coordinates must be added manually
This is done by uncommenting the line below and changing the index, 'longitude' and 'latitude' with the actual numbers. Lists are 0-indexed such that if the missing port is the first one, the index is 0.

In [7]:
#coordinates.insert(index,[longitude,latitude])

# Specifying properties needed for routing

In the following cells we are specifying speed and draught for the vessels as well as graphing, time resolution, how detailed the routing should be and how much other vessel data should be weighed for the routing. 

<p style="color:#9D59F4; font-size:15pt; font-weight:bold"> Make a list for the speeds to simulate with for each vessel  </p> 
<p> If the speed is set to None, the maximum speed for the vessel is chosen. </p> 

In [8]:
speeds = [None] * len(mmsis)
# Or:
# speeds = [10, 10] #Here, the speed for both vessels is set to 10 knots
speeds

[None, None, None, None]

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> List of draught for the different vessels </p> 
<p> If they are set to None, the draught will be found in the data about the vessel particulars. </p> 

In [9]:
draughts = [None] * len(mmsis)
# Or:
# draughts = [7, 9] #here the draught for the first ship is 7m and for the second ship it is 9 m
draughts

[None, None, None, None]

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose grapfing types for the vessels </p> 
<p> This is also found from the vessel particulars, but in the incidences where it cannot be found, it needs to be specified. Possible graph types are passenger, fishing, cargo, tanker and tug. </p> 

In [10]:
graphs = [None] * len(mmsis)
# Or:
# graphs = ['cargo', 'tug']
graphs

[None, None, None, None]

### Make a dataframe of the mmsis, speeds, draughts and graphs
This is used as input in the function finding routes and emissions.

In [11]:
zipped_lists = list(zip(speeds, draughts, graphs))
df_input_properties = pd.DataFrame(zipped_lists, index = mmsis, columns = ["speeds","draughts","graphs"])


In [12]:
df_input_properties

,speeds,draughts,graphs
212552000,None,None,None
319009700,None,None,None
352297000,None,None,None
477630400,None,None,None


<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose the time resolution for the graphing </p> 
<p> The default value is every 60th minute. Choose a value between 5 and 240. </p> 

In [13]:
time_resolution = 60

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Decide how fine/coarse the routing should be </p> 

 * 0 = Coarse (fast)
 * 1 = Coarse + refined
 * 2 = Fine (slow)

In [14]:
detailed_routing = 1

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> How much should popular routes be weighted in the routing? </p> 
<p> Choose a value between 0 and 0.95. Default is 0.3. </p> 

In [15]:
cost_density = 0.3

## Finding paths and emissions for each vessel

Using a premade function called emissions_multiple_vessels that takes the coordinates chosen and the df_input_properties as arguments. OBS, this might take a while. 

In [16]:
%%time
df_emissions, df_summed_emissions = emissions_and_paths(
    coordinates,
    df_input_properties,
    time_resolution,
    detailed_routing,
    cost_density
)

Path finding for vessels with mmmsis 212552000 352297000 477630400 is done. 
Path finding for vessels with mmmsi 319009700 is done. 
CPU times: user 12.5 s, sys: 517 ms, total: 13 s
Wall time: 2min 6s


### Summed emissions for all vessels compared

The vessel at the top is the one that emitts the least amount of CO2.

In [17]:
df_summed_emissions

,emissions_CO2,emissions_CO,emissions_SOX,emissions_N2O,emissions_NOX,emissions_PM,emissions_CH4
ssvid,,,,,,,
319009700,7.037164e+07,58686.188354,60934.823105,3208.464907,8.029654e+05,21466.543621,1092.698865
212552000,8.800323e+07,72827.149552,54800.454699,4008.237787,1.764451e+06,25442.076641,1354.540793
477630400,1.535896e+08,131080.440648,95641.045622,7051.341309,2.868777e+06,45028.377820,2468.830237
352297000,1.765687e+08,164356.748345,109976.947472,8806.317347,4.521256e+06,56281.982169,3099.326742


## Show the paths
This one is for the vessel that uses the least amount of CO2

In [18]:
mmsi0 = df_summed_emissions.index[0]
print(f"Emissions for vessel with mmsi {mmsi0}")
plot_df(df_emissions['emissions_on_path'].loc[mmsi0])

Emissions for vessel with mmsi 319009700


{"initialViewState": {"latitude": 55.32978865962629, "longitude": 8.958050188646475, "zoom": 3}, "layers": [{"@@type": "PathLayer", "data": [{"path": [[10.5908203125, 59.83154296875], [10.55419921875, 59.75830078125], [10.634765625, 59.677734375], [10.60400390625, 59.57666015625], [10.56884765625, 59.43603515625], [10.62158203125, 59.23828125], [10.669921875, 59.0185546875], [10.74951171875, 58.81103515625], [10.82763671875, 58.61083984375], [10.92529296875, 58.41552734375], [11.06689453125, 58.21533203125], [11.1796875, 58.0078125], [11.28662109375, 57.80029296875], [11.41845703125, 57.60009765625], [11.45751953125, 57.40478515625], [11.60888671875, 57.20458984375], [11.74560546875, 57.00927734375], [11.81884765625, 56.80908203125], [11.70166015625, 56.61376953125], [11.50146484375, 56.41357421875], [11.31591796875, 56.22802734375], [11.1181640625, 56.06103515625], [10.92041015625, 55.87646484375], [10.79736328125, 55.6787109375], [10.92041015625, 55.48095703125], [11.06689453125, 55.29541015625], [11.03271484375, 55.09521484375], [10.87646484375, 54.89990234375], [10.73583984375, 54.68994140625], [10.5029296875, 54.55078125], [10.292619977678571, 54.476492745535715], [10.18798828125, 54.39697265625], [10.01953125, 54.3603515625], [9.8876953125, 54.3603515625], [9.755859375, 54.345703125], [9.66064453125, 54.30908203125], [9.60205078125, 54.22119140625], [9.47021484375, 54.16259765625], [9.34716796875, 54.12744140625], [9.29443359375, 54.03076171875], [9.24169921875, 53.93408203125], [9.0966796875, 53.85498046875], [8.89892578125, 53.85498046875], [8.701171875, 53.9208984375], [8.525390625, 54.019775390625], [8.27197265625, 54.11865234375], [7.97607421875, 54.100341796875], [7.71240234375, 54.034423828125], [7.44873046875, 53.990478515625], [7.241559709821429, 53.91148158482143], [7.00927734375, 53.848702566964285], [6.70166015625, 53.817313058035715], [6.38232421875, 53.82275390625], [6.064453125, 53.782784598214285], [5.7907936789772725, 53.72713955965909], [5.59423828125, 53.61767578125], [5.38330078125, 53.56494140625], [5.1416015625, 53.51806640625], [4.866098257211538, 53.44257061298077], [4.63623046875, 53.31787109375], [4.482421875, 53.1298828125], [4.317626953125, 52.9541015625], [4.18701171875, 52.76611328125], [4.29443359375, 52.59033203125], [4.49462890625, 52.49267578125], [4.64501953125, 52.45751953125], [4.74609375, 52.40478515625]]}], "getColor": "@@=[0,0,0, 150]", "id": "97c88a95-7f3d-4750-89cd-bd82fc9f18c2", "widthMinPixels": 2}, {"@@type": "PointCloudLayer", "autoHighlight": true, "data": [{"distance_from_port_m": 12981.237303026408, "distance_from_shore_m": 1437.4740912181348, "draught_recent_m": 3.15, "emissions_CH4": 5.377594809021392, "emissions_CO": 290.3901196871552, "emissions_CO2": 358785.7384336076, "emissions_N2O": 16.132784427064177, "emissions_NOX": 4321.0786417314885, "emissions_PM": 107.55189618042786, "emissions_SOX": 310.42290411421936, "in_a_eca": 1, "in_a_river": 0, "lat": 59.83154296875, "lon": 10.5908203125, "speed_knots": 7.192255956870812, "ssvid": 319009700, "timestamp": "2022-01-01 01:00:00", "total_power_demand_kw": 537.7594809021393}, {"distance_from_port_m": 20730.83339624317, "distance_from_shore_m": 1072.7689531880287, "draught_recent_m": 3.15, "emissions_CH4": 5.350110039309836, "emissions_CO": 285.4344192318942, "emissions_CO2": 301985.74649729143, "emissions_N2O": 14.02527509827459, "emissions_NOX": 3812.466390383601, "emissions_PM": 95.43045781674006, "emissions_SOX": 261.219263683064, "in_a_eca": 1, "in_a_river": 0, "lat": 59.75830078125, "lon": 10.55419921875, "speed_knots": 6.643993587816856, "ssvid": 319009700, "timestamp": "2022-01-01 02:00:00", "total_power_demand_kw": 451.43730470712984}, {"distance_from_port_m": 27338.90678552361, "distance_from_shore_m": 0.0, "draught_recent_m": 3.15, "emissions_CH4": 6.236035729845302, "emissions_CO": 282.22517112290046, "emissions_CO2": 182750.21335904003, "emissions_N2O": 9.999930518416779, "emissions_NOX": 2841.0165651112493, "e

In [19]:
mmsi1 = df_summed_emissions.index[1]
print(f"Emissions for vessel with mmsi {mmsi1}")
plot_df(df_emissions['emissions_on_path'].loc[mmsi1])

Emissions for vessel with mmsi 212552000


{"initialViewState": {"latitude": 55.016132074601515, "longitude": 8.765142153754852, "zoom": 3}, "layers": [{"@@type": "PathLayer", "data": [{"path": [[10.579833984375, 59.74365234375], [10.594482421875, 59.52392578125], [10.601806640625, 59.26025390625], [10.634765625, 58.99658203125], [10.755615234375, 58.73291015625], [10.896439985795455, 58.48122336647727], [11.03759765625, 58.2275390625], [11.18408203125, 57.97385475852273], [11.275634765625, 57.72216796875], [11.427778764204545, 57.47048117897727], [11.60888671875, 57.216796875], [11.763361150568182, 56.96311257102273], [11.795321377840908, 56.72341086647727], [11.567604758522727, 56.47971413352273], [11.31591796875, 56.231689453125], [11.064231178977273, 56.02028586647727], [10.836514559659092, 55.77658913352273], [10.868474786931818, 55.53688742897727], [11.060236150568182, 55.29319069602273], [11.011962890625, 55.04150390625], [10.83251953125, 54.78981711647727], [10.56884765625, 54.57066127232143], [10.30517578125, 54.45263671875], [10.14697265625, 54.37353515625], [10.01953125, 54.349365234375], [9.86572265625, 54.36767578125], [9.721330915178571, 54.30698939732143], [9.59326171875, 54.24169921875], [9.457659040178571, 54.17515345982143], [9.33837890625, 54.0966796875], [9.27685546875, 53.97802734375], [9.193987165178571, 53.89892578125], [8.95458984375, 53.85791015625], [8.67919921875, 53.91650390625], [8.338099888392858, 53.98681640625], [7.899748149671052, 54.02844880756579], [7.49267578125, 54.00146484375], [7.09716796875, 53.93310546875], [6.690095600328948, 53.8781095805921], [6.295812270220588, 53.82396024816177], [5.899079975328948, 53.7462736430921], [5.504796645220588, 53.66627412683823], [5.20751953125, 53.54736328125], [5.069405691964286, 53.365304129464285], [5.194963727678571, 53.264857700892854], [5.33056640625, 53.20458984375], [5.33056640625, 53.09912109375], [5.33935546875, 53.00537109375], [5.361328125, 52.93212890625], [5.35693359375, 52.83544921875], [5.483747209821429, 52.737513950892854], [5.52978515625, 52.646484375], [5.433523995535714, 52.517787388392854], [5.2734375, 52.470703125], [5.11962890625, 52.40478515625]]}], "getColor": "@@=[0,0,0, 150]", "id": "6783307d-fc47-4d9a-92ef-e569f8556be1", "widthMinPixels": 2}, {"@@type": "PointCloudLayer", "autoHighlight": true, "data": [{"distance_from_port_m": 21460.245713522854, "distance_from_shore_m": 0.0, "draught_recent_m": 6.34, "emissions_CH4": 33.844, "emissions_CO": 1827.576, "emissions_CO2": 2233395.2, "emissions_N2O": 101.532, "emissions_NOX": 44650.904, "emissions_PM": 643.036, "emissions_SOX": 1391.004, "in_a_eca": 1, "in_a_river": 0, "lat": 59.74365234375, "lon": 10.579833984375, "speed_knots": 13.271320267131218, "ssvid": 212552000, "timestamp": "2022-01-01 01:00:00", "total_power_demand_kw": 3384.4}, {"distance_from_port_m": 44532.81786880429, "distance_from_shore_m": 3778.839325134989, "draught_recent_m": 6.34, "emissions_CH4": 33.844, "emissions_CO": 1827.576, "emissions_CO2": 2233395.2, "emissions_N2O": 101.532, "emissions_NOX": 44650.904, "emissions_PM": 643.036, "emissions_SOX": 1391.004, "in_a_eca": 1, "in_a_river": 0, "lat": 59.52392578125, "lon": 10.594482421875, "speed_knots": 15.688087495160639, "ssvid": 212552000, "timestamp": "2022-01-01 02:00:00", "total_power_demand_kw": 3384.4}, {"distance_from_port_m": 73464.95490825715, "distance_from_shore_m": 5467.210380047116, "draught_recent_m": 6.34, "emissions_CH4": 33.844, "emissions_CO": 1827.576, "emissions_CO2": 2233395.2, "emissions_N2O": 101.532, "emissions_NOX": 44650.904, "emissions_PM": 643.036, "emissions_SOX": 1391.004, "in_a_eca": 1, "in_a_river": 0, "lat": 59.26025390625, "lon": 10.601806640625, "speed_knots": 16.3333333333333, "ssvid": 212552000, "timestamp": "2022-01-01 03:00:00", "total_power_demand_kw": 3384.4}, {"distance_from_port_m": 102515.89919226305, "distance_from_shore_m": 13186.996529341479, "draught_recent_m": 6.34, "emissions_CH4": 33.844, "emissions_CO": 1827.576, "emissions_CO2": 2233395.2, "emissi

If you are comparing more than two vessels and want to see the paths of the other ones, uncomment the line underneath. If there are more than two vessels you can also change the index to see the other (OBS, zero indexed). 

In [20]:
#plot_df(df_emissions['emissions_on_path'].loc[df_summed_emissions.index[2]])